# Numerical Benchmarks

In [1]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_maps import *
from quantum_tools import *
from experiments import *
#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=3)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
def generate_spam_data(spam_target, noise = 0):
    n = int(np.log2(spam_target.d))
    
    inputs_spam, _ = generate_pauliInput_circuits(n)

    U_prep = inputs_spam
    N_spam = U_prep.shape[0]

    state = tf.repeat(spam_target.init[None,:,:], N_spam, axis=0)
    state = apply_unitary(state, U_prep)
    targets_spam = measurement(state, povm = spam_target.povm)
    targets_spam = targets_spam + tf.cast(tf.random.normal(targets_spam.shape, 0, noise), dtype = precision)
    
    return inputs_spam, targets_spam


def generate_map_data(kraus_target, noise = 0):
    n = int(np.log2(kraus_target.d))
    inputs_map, _ = generate_pauli_circuits(n = n, 
                                            circuit_target=None, 
                                            N = None, 
                                            trace=False)
    U_prep, U_basis = inputs_map
    N_map = U_prep.shape[0]
    
    state = tf.repeat(tf.expand_dims(spam_target.init, axis=0), N_map, axis=0)
    state = apply_unitary(state, U_prep)
    state = kraus_target.apply_map(state)
    targets_map = measurement(state, U_basis, spam_target.povm)
    targets_map = targets_map + tf.cast(tf.random.normal(targets_map.shape, 0, noise), dtype = precision)
    
    return inputs_map, targets_map

## Collapse of SPAM optimization

In [24]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 2
d = 2**n
rank = d

spam_target = SPAM(d=d, 
                   init = "ideal",
                   povm = "ideal",
                   )

### Generate Synthetic Data

In [25]:
inputs_spam, targets_spam = generate_spam_data(spam_target)
N_spam = inputs_spam.shape[0]

### Fit SPAM model, no Pre-Training

In [13]:
np.random.seed(43)
random.seed(43)
tf.random.set_seed(43)

spam_model = SPAM(d=d,
                  optimizer = tf.optimizers.Adam(learning_rate=0.01))

spam_model.train(inputs = inputs_spam,
                 targets = targets_spam,
                 num_iter = 1000,
                 N = N_spam
                 )

  0%|          | 0/1000 [00:00<?, ?it/s]

0 0.8921902649442504
1 0.8791598613811485
2 0.8664378444021044
3 0.8540343076056153
4 0.8419629923787567
5 0.8302745546766239
6 0.8189254316461283
7 0.8079260217838797
8 0.797328867453269
9 0.7870964791473015
10 0.777252467281691
11 0.7678012329386679
12 0.7587422764817365
13 0.7500702533673002
14 0.7417746797010677
15 0.7338403029673441
16 0.7262488917928419
17 0.7189027999203573
18 0.7119036809497677
19 0.7051607061504508
20 0.698659058733764
21 0.6923715001368348
22 0.686269955117023
23 0.6803124724279446
24 0.6744751479178335
25 0.6685125168635264
26 0.6625511847186242
27 0.6565805147614103
28 0.6505763580483962
29 0.6445173936169589
30 0.6383315697365317
31 0.6320002069949808
32 0.6255589484899267
33 0.6190020285950987
34 0.6123305431961066
35 0.6055321927646852
36 0.5985269314117969
37 0.5914369933423573
38 0.584354221313732
39 0.5772045325834525
40 0.5699881035740305
41 0.5626968709831999
42 0.5553200487824674
43 0.5479018421513429
44 0.5406513321294365
45 0.5333543059059592
46 

348 0.0015876286600823154
349 0.0015708913235540818
350 0.0015543946231788024
351 0.0015381343414898033
352 0.001522106365904954
353 0.0015063066799949558
354 0.0014907313551195392
355 0.0014753765424396434
356 0.001460238465348555
357 0.0014453134123963915
358 0.001430597730811323
359 0.0014160878207449149
360 0.0014017801303755812
361 0.0013876711520023146
362 0.0013737574192336178
363 0.0013600355053461927
364 0.0013465020228343465
365 0.0013331536241198115
366 0.0013199870033330794
367 0.0013069988990203857
368 0.0012941860975886942
369 0.0012815454372639462
370 0.0012690738123173487
371 0.0012567681773128726
372 0.0012446255511396867
373 0.0012326430206174334
374 0.001220817743504387
375 0.0012091469507725275
376 0.001197627948074157
377 0.0011862581163608614
378 0.0011750349116642143
379 0.0011639558640857517
380 0.0011530185760675307
381 0.001142220720039083
382 0.0011315600355436058
383 0.0011210343259504705
384 0.0011106414548597173
385 0.0011003793422912208
386 0.001090245960

665 0.0001735402113563271
666 0.00017268636408901812
667 0.00017183802017576688
668 0.00017099513491612738
669 0.000170157664047628
670 0.00016932556374858848
671 0.00016849879063978522
672 0.00016767730178572472
673 0.0001668610546942527
674 0.0001660500073155265
675 0.0001652441180389652
676 0.00016444334568941544
677 0.00016364764952132111
678 0.0001628569892120496
679 0.0001620713248538926
680 0.00016129061694522038
681 0.00016051482638076699
682 0.0001597439144416271
683 0.00015897784278510034
684 0.00015821657343413953
685 0.0001574600687680136
686 0.00015670829151252685
687 0.00015596120473190575
688 0.00015521877182104966
689 0.00015448095649893564
690 0.00015374772280386058
691 0.0001530190350884746
692 0.0001522948580173824
693 0.00015157515656496725
694 0.00015085989601453504
695 0.00015014904195782854
696 0.00014944256029593512
697 0.00014874041723971485
698 0.00014804257931133342
699 0.00014734901334494848
700 0.00014665968648790175
701 0.00014597456620117665
702 0.0001452

978 4.866483464564842e-05
979 4.849913406703707e-05
980 4.8334148550515586e-05
981 4.816987429590819e-05
982 4.8006307527217275e-05
983 4.7843444492757436e-05
984 4.768128146479113e-05
985 4.751981473942832e-05
986 4.7359040636313364e-05
987 4.7198955498505205e-05
988 4.7039555692172285e-05
989 4.688083760658388e-05
990 4.672279765360881e-05
991 4.6565432267755674e-05
992 4.640873790585695e-05
993 4.625271104693287e-05
994 4.6097348191959155e-05
995 4.5942645863702894e-05
996 4.578860060668322e-05
997 4.5635208986863736e-05
998 4.5482467591567404e-05
999 4.533037302947756e-05


In [14]:
print(spam_target.init.numpy())
print(spam_model.init.numpy())
print("--------")
print(spam_target.povm.numpy())
print(spam_model.povm.numpy())

[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
[[ 1.965e-03+0.000e+00j -7.086e-04-3.434e-02j -5.023e-05-1.687e-05j
   1.188e-03+1.347e-04j]
 [-7.086e-04+3.434e-02j  9.967e-01+0.000e+00j -8.379e-04-6.996e-04j
   1.668e-03+3.468e-02j]
 [-5.023e-05+1.687e-05j -8.379e-04+6.996e-04j  7.517e-06+0.000e+00j
  -3.293e-05-3.832e-05j]
 [ 1.188e-03-1.347e-04j  1.668e-03-3.468e-02j -3.293e-05+3.832e-05j
   1.284e-03+0.000e+00j]]
--------
[[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]

 [[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]

 [[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]

 [[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0

In [15]:
np.random.seed(43)
random.seed(43)
tf.random.set_seed(43)

spam_model = SPAM(d=d,
                  optimizer = tf.optimizers.Adam(learning_rate=0.01))

spam_model.train(inputs = inputs_spam,
                 targets = targets_spam,
                 num_iter = 1000,
                 N = N_spam
                 )

  0%|          | 0/1000 [00:00<?, ?it/s]

0 0.8921902649442504
1 0.8791598613811485
2 0.8664378444021044
3 0.8540343076056153
4 0.8419629923787567
5 0.8302745546766239
6 0.8189254316461283
7 0.8079260217838797
8 0.797328867453269
9 0.7870964791473015
10 0.777252467281691
11 0.7678012329386679
12 0.7587422764817365
13 0.7500702533673002
14 0.7417746797010677
15 0.7338403029673441
16 0.7262488917928419
17 0.7189027999203573
18 0.7119036809497677
19 0.7051607061504508
20 0.698659058733764
21 0.6923715001368348
22 0.686269955117023
23 0.6803124724279446
24 0.6744751479178335
25 0.6685125168635264
26 0.6625511847186242
27 0.6565805147614103
28 0.6505763580483962
29 0.6445173936169589
30 0.6383315697365317
31 0.6320002069949808
32 0.6255589484899267
33 0.6190020285950987
34 0.6123305431961066
35 0.6055321927646852
36 0.5985269314117969
37 0.5914369933423573
38 0.584354221313732
39 0.5772045325834525
40 0.5699881035740305
41 0.5626968709831999
42 0.5553200487824674
43 0.5479018421513429
44 0.5406513321294365
45 0.5333543059059592
46 

349 0.0015708913235540818
350 0.0015543946231788024
351 0.0015381343414898033
352 0.001522106365904954
353 0.0015063066799949558
354 0.0014907313551195392
355 0.0014753765424396434
356 0.001460238465348555
357 0.0014453134123963915
358 0.001430597730811323
359 0.0014160878207449149
360 0.0014017801303755812
361 0.0013876711520023146
362 0.0013737574192336178
363 0.0013600355053461927
364 0.0013465020228343465
365 0.0013331536241198115
366 0.0013199870033330794
367 0.0013069988990203857
368 0.0012941860975886942
369 0.0012815454372639462
370 0.0012690738123173487
371 0.0012567681773128726
372 0.0012446255511396867
373 0.0012326430206174334
374 0.001220817743504387
375 0.0012091469507725275
376 0.001197627948074157
377 0.0011862581163608614
378 0.0011750349116642143
379 0.0011639558640857517
380 0.0011530185760675307
381 0.001142220720039083
382 0.0011315600355436058
383 0.0011210343259504705
384 0.0011106414548597173
385 0.0011003793422912208
386 0.0010902459607476944
387 0.001080239331

667 0.00017183802017576688
668 0.00017099513491612738
669 0.000170157664047628
670 0.00016932556374858848
671 0.00016849879063978522
672 0.00016767730178572472
673 0.0001668610546942527
674 0.0001660500073155265
675 0.0001652441180389652
676 0.00016444334568941544
677 0.00016364764952132111
678 0.0001628569892120496
679 0.0001620713248538926
680 0.00016129061694522038
681 0.00016051482638076699
682 0.0001597439144416271
683 0.00015897784278510034
684 0.00015821657343413953
685 0.0001574600687680136
686 0.00015670829151252685
687 0.00015596120473190575
688 0.00015521877182104966
689 0.00015448095649893564
690 0.00015374772280386058
691 0.0001530190350884746
692 0.0001522948580173824
693 0.00015157515656496725
694 0.00015085989601453504
695 0.00015014904195782854
696 0.00014944256029593512
697 0.00014874041723971485
698 0.00014804257931133342
699 0.00014734901334494848
700 0.00014665968648790175
701 0.00014597456620117665
702 0.0001452936202594229
703 0.00014461681675013896
704 0.0001439

982 4.8006307527217275e-05
983 4.7843444492757436e-05
984 4.768128146479113e-05
985 4.751981473942832e-05
986 4.7359040636313364e-05
987 4.7198955498505205e-05
988 4.7039555692172285e-05
989 4.688083760658388e-05
990 4.672279765360881e-05
991 4.6565432267755674e-05
992 4.640873790585695e-05
993 4.625271104693287e-05
994 4.6097348191959155e-05
995 4.5942645863702894e-05
996 4.578860060668322e-05
997 4.5635208986863736e-05
998 4.5482467591567404e-05
999 4.533037302947756e-05


In [11]:
print(spam_target.init.numpy())
print(spam_model.init.numpy())
print("--------")
print(spam_target.povm.numpy())
print(spam_model.povm.numpy())

[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
--------
[[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]

 [[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]

 [[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]

 [[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 1.+0.j]]]
[[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]

 [[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
  [0.+0.j 0.+0.j 0.+0.j 0.+

## Retrieve SPAM and Map, No Noise

In [6]:
n = 3
d = 2**n
rank = d
c1 = 0.9
c2 = 0.9

### Retrieve Map without SPAM

In [5]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

model_list = []
for i in range(1):
    
    #make random spam model
    spam_target = SPAM(d=d)

    #add ideal part
    init_target = c1*init_ideal(d) + (1-c1)*spam_target.init
    povm_target = c2*povm_ideal(d) + (1-c2)*spam_target.povm

    spam_target = SPAM(d=d,
                       init = init_target,
                       povm = povm_target)
    
    # random kraus model with unitary part
    kraus_target = KrausMap(
                            U = generate_unitary(d=d), 
                            c = 0.5, 
                            d = d, 
                            rank = rank-1,
                            spam = spam_target
                           )
    
    #inputs_spam, targets_spam = generate_spam_data(spam_target)
    inputs_map, targets_map = generate_map_data(kraus_target)
    
    
    kraus_model = KrausMap(
                           d = d, 
                           rank = rank,
                           spam = None,
                           )

    model = ModelQuantumMap(
                            q_map = kraus_model,
                            loss = probs_loss,
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            )

    model.train(inputs = inputs_map,
                targets = targets_map,
                num_iter = 1000,
                N = 1000,
                )
    
    model_list.append(model)

KeyboardInterrupt: 

In [4]:
channel_fidelity(model_list[0].q_map, kraus_target).numpy()

0.9740266064506574

### Retrieve SPAM and Map

In [7]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

model_list = []

for i in range(1):    
    #make random spam model
    spam_target = SPAM(d=d)

    #add ideal part
    init_target = c1*init_ideal(d) + (1-c1)*spam_target.init
    povm_target = c2*povm_ideal(d) + (1-c2)*spam_target.povm

    spam_target = SPAM(d=d,
                       init = init_target,
                       povm = povm_target)
    
    # random kraus model with unitary part
    kraus_target = KrausMap(
                            U = generate_unitary(d=d), 
                            c = 0.5, 
                            d = d, 
                            rank = rank-1,
                            spam = spam_target
                           )
    
    inputs_spam, targets_spam = generate_spam_data(spam_target)
    inputs_map, targets_map = generate_map_data(kraus_target)

    #train spam model
    spam_model = SPAM(d=d,
    #                  use_corr_mat=True,
                      optimizer = tf.optimizers.Adam(learning_rate=0.01))
    
    spam_model.pretrain(targets = [init_ideal(d), povm_ideal(d)],
                        num_iter = 1000,
                        verbose = False
                        )
    
    spam_model.train(inputs = inputs_spam,
                     targets = targets_spam,
                     num_iter = 1000,
                     verbose = False,
                    )

    #train kraus model
    kraus_model = KrausMap(
                           d = d, 
                           rank = rank,
                           spam = spam_model,
                           )

    model = ModelQuantumMap(
                            q_map = kraus_model,
                            loss = probs_loss,
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            )

    model.train(inputs = inputs_map,
                targets = targets_map,
                #inputs_val = inputs_map,
                #targets_val = targets_map,
                num_iter = 1000,
                N = 1000
                )
    
    model_list.append(model)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Step:0, train: 0.21585, val: 0.00000
Step:1, train: 0.21835, val: 0.00000
Step:2, train: 0.21692, val: 0.00000
Step:3, train: 0.20963, val: 0.00000
Step:4, train: 0.20888, val: 0.00000
Step:5, train: 0.20416, val: 0.00000
Step:6, train: 0.20488, val: 0.00000
Step:7, train: 0.19507, val: 0.00000
Step:8, train: 0.20048, val: 0.00000
Step:9, train: 0.19768, val: 0.00000
Step:10, train: 0.19977, val: 0.00000
Step:11, train: 0.19316, val: 0.00000
Step:12, train: 0.18607, val: 0.00000
Step:13, train: 0.18815, val: 0.00000
Step:14, train: 0.18289, val: 0.00000
Step:15, train: 0.17768, val: 0.00000
Step:16, train: 0.17832, val: 0.00000
Step:17, train: 0.18328, val: 0.00000
Step:18, train: 0.17340, val: 0.00000
Step:19, train: 0.17232, val: 0.00000
Step:20, train: 0.16845, val: 0.00000
Step:21, train: 0.16603, val: 0.00000
Step:22, train: 0.16304, val: 0.00000
Step:23, train: 0.16001, val: 0.00000
Step:24, train: 0.15945, val: 0.00000
Step:25, train: 0.15769, val: 0.00000
Step:26, train: 0.1542

Step:214, train: 0.01085, val: 0.00000
Step:215, train: 0.01073, val: 0.00000
Step:216, train: 0.01057, val: 0.00000
Step:217, train: 0.01066, val: 0.00000
Step:218, train: 0.01034, val: 0.00000
Step:219, train: 0.01021, val: 0.00000
Step:220, train: 0.01022, val: 0.00000
Step:221, train: 0.01045, val: 0.00000
Step:222, train: 0.01040, val: 0.00000
Step:223, train: 0.01001, val: 0.00000
Step:224, train: 0.01004, val: 0.00000
Step:225, train: 0.01019, val: 0.00000
Step:226, train: 0.00974, val: 0.00000
Step:227, train: 0.00988, val: 0.00000
Step:228, train: 0.01010, val: 0.00000
Step:229, train: 0.01020, val: 0.00000
Step:230, train: 0.00972, val: 0.00000
Step:231, train: 0.00980, val: 0.00000
Step:232, train: 0.00991, val: 0.00000
Step:233, train: 0.00961, val: 0.00000
Step:234, train: 0.00952, val: 0.00000
Step:235, train: 0.00971, val: 0.00000
Step:236, train: 0.00925, val: 0.00000
Step:237, train: 0.00950, val: 0.00000
Step:238, train: 0.00904, val: 0.00000
Step:239, train: 0.00948,

Step:426, train: 0.00361, val: 0.00000
Step:427, train: 0.00346, val: 0.00000
Step:428, train: 0.00355, val: 0.00000
Step:429, train: 0.00347, val: 0.00000
Step:430, train: 0.00344, val: 0.00000
Step:431, train: 0.00345, val: 0.00000
Step:432, train: 0.00336, val: 0.00000
Step:433, train: 0.00323, val: 0.00000
Step:434, train: 0.00340, val: 0.00000
Step:435, train: 0.00338, val: 0.00000
Step:436, train: 0.00341, val: 0.00000
Step:437, train: 0.00327, val: 0.00000
Step:438, train: 0.00318, val: 0.00000
Step:439, train: 0.00325, val: 0.00000
Step:440, train: 0.00314, val: 0.00000
Step:441, train: 0.00319, val: 0.00000
Step:442, train: 0.00303, val: 0.00000
Step:443, train: 0.00312, val: 0.00000
Step:444, train: 0.00326, val: 0.00000
Step:445, train: 0.00311, val: 0.00000
Step:446, train: 0.00319, val: 0.00000
Step:447, train: 0.00296, val: 0.00000
Step:448, train: 0.00308, val: 0.00000
Step:449, train: 0.00305, val: 0.00000
Step:450, train: 0.00310, val: 0.00000
Step:451, train: 0.00301,

Step:638, train: 0.00078, val: 0.00000
Step:639, train: 0.00079, val: 0.00000
Step:640, train: 0.00079, val: 0.00000
Step:641, train: 0.00074, val: 0.00000
Step:642, train: 0.00077, val: 0.00000
Step:643, train: 0.00072, val: 0.00000
Step:644, train: 0.00072, val: 0.00000
Step:645, train: 0.00072, val: 0.00000
Step:646, train: 0.00074, val: 0.00000
Step:647, train: 0.00072, val: 0.00000
Step:648, train: 0.00070, val: 0.00000
Step:649, train: 0.00072, val: 0.00000
Step:650, train: 0.00071, val: 0.00000
Step:651, train: 0.00071, val: 0.00000
Step:652, train: 0.00070, val: 0.00000
Step:653, train: 0.00068, val: 0.00000
Step:654, train: 0.00067, val: 0.00000
Step:655, train: 0.00069, val: 0.00000
Step:656, train: 0.00066, val: 0.00000
Step:657, train: 0.00068, val: 0.00000
Step:658, train: 0.00069, val: 0.00000
Step:659, train: 0.00066, val: 0.00000
Step:660, train: 0.00066, val: 0.00000
Step:661, train: 0.00063, val: 0.00000
Step:662, train: 0.00064, val: 0.00000
Step:663, train: 0.00061,

Step:850, train: 0.00017, val: 0.00000
Step:851, train: 0.00017, val: 0.00000
Step:852, train: 0.00016, val: 0.00000
Step:853, train: 0.00016, val: 0.00000
Step:854, train: 0.00017, val: 0.00000
Step:855, train: 0.00017, val: 0.00000
Step:856, train: 0.00017, val: 0.00000
Step:857, train: 0.00016, val: 0.00000
Step:858, train: 0.00016, val: 0.00000
Step:859, train: 0.00016, val: 0.00000
Step:860, train: 0.00016, val: 0.00000
Step:861, train: 0.00016, val: 0.00000
Step:862, train: 0.00016, val: 0.00000
Step:863, train: 0.00016, val: 0.00000
Step:864, train: 0.00015, val: 0.00000
Step:865, train: 0.00016, val: 0.00000
Step:866, train: 0.00015, val: 0.00000
Step:867, train: 0.00015, val: 0.00000
Step:868, train: 0.00015, val: 0.00000
Step:869, train: 0.00015, val: 0.00000
Step:870, train: 0.00015, val: 0.00000
Step:871, train: 0.00014, val: 0.00000
Step:872, train: 0.00014, val: 0.00000
Step:873, train: 0.00015, val: 0.00000
Step:874, train: 0.00014, val: 0.00000
Step:875, train: 0.00015,

In [8]:
channel_fidelity(model_list[0].q_map, kraus_target).numpy()

0.9983357840232873

### Retrieve SPAM (Corruption Matrix) and Map

In [13]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

model_list = []

for i in range(1):    
    #make random spam model
    spam_target = SPAM(d=d)

    #add ideal part
    init_target = c1*init_ideal(d) + (1-c1)*spam_target.init
    povm_target = c2*povm_ideal(d) + (1-c2)*spam_target.povm

    spam_target = SPAM(d=d,
                       init = init_target,
                       povm = povm_target)
    
    # random kraus model with unitary part
    kraus_target = KrausMap(
                            U = generate_unitary(d=d), 
                            c = 0.5, 
                            d = d, 
                            rank = rank-1,
                            spam = spam_target
                           )
    
    inputs_spam, targets_spam = generate_spam_data(spam_target)
    inputs_map, targets_map = generate_map_data(kraus_target)

    #train spam model
    spam_model = SPAM(d=d,
                      use_corr_mat=True,
                      optimizer = tf.optimizers.Adam(learning_rate=0.01))
    
    spam_model.pretrain(targets = [init_ideal(d), povm_ideal(d)],
                        num_iter = 1000,
                        verbose = False
                        )
    
    spam_model.train(inputs = inputs_spam,
                     targets = targets_spam,
                     num_iter = 1000,
                     verbose = False,
                    )

    #train kraus model
    kraus_model = KrausMap(
                           d = d, 
                           rank = rank,
                           spam = spam_model,
                           )

    model = ModelQuantumMap(
                            q_map = kraus_model,
                            loss = probs_loss,
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            )

    model.train(inputs = inputs_map,
                targets = targets_map,
                #inputs_val = inputs_map,
                #targets_val = targets_map,
                num_iter = 1000,
                N = 1000
                )
    
    model_list.append(model)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Step:0, train: 0.21583, val: 0.00000
Step:1, train: 0.20838, val: 0.00000
Step:2, train: 0.21248, val: 0.00000
Step:3, train: 0.20878, val: 0.00000
Step:4, train: 0.20796, val: 0.00000
Step:5, train: 0.20468, val: 0.00000
Step:6, train: 0.19952, val: 0.00000
Step:7, train: 0.19993, val: 0.00000
Step:8, train: 0.19272, val: 0.00000
Step:9, train: 0.19831, val: 0.00000
Step:10, train: 0.19651, val: 0.00000
Step:11, train: 0.19280, val: 0.00000
Step:12, train: 0.18566, val: 0.00000
Step:13, train: 0.18102, val: 0.00000
Step:14, train: 0.18670, val: 0.00000
Step:15, train: 0.18091, val: 0.00000
Step:16, train: 0.17295, val: 0.00000
Step:17, train: 0.17344, val: 0.00000
Step:18, train: 0.17499, val: 0.00000
Step:19, train: 0.16974, val: 0.00000
Step:20, train: 0.16606, val: 0.00000
Step:21, train: 0.16318, val: 0.00000
Step:22, train: 0.16455, val: 0.00000
Step:23, train: 0.16410, val: 0.00000
Step:24, train: 0.16373, val: 0.00000
Step:25, train: 0.15886, val: 0.00000
Step:26, train: 0.1575

Step:213, train: 0.01206, val: 0.00000
Step:214, train: 0.01188, val: 0.00000
Step:215, train: 0.01192, val: 0.00000
Step:216, train: 0.01192, val: 0.00000
Step:217, train: 0.01167, val: 0.00000
Step:218, train: 0.01195, val: 0.00000
Step:219, train: 0.01176, val: 0.00000
Step:220, train: 0.01178, val: 0.00000
Step:221, train: 0.01143, val: 0.00000
Step:222, train: 0.01180, val: 0.00000
Step:223, train: 0.01159, val: 0.00000
Step:224, train: 0.01149, val: 0.00000
Step:225, train: 0.01109, val: 0.00000
Step:226, train: 0.01096, val: 0.00000
Step:227, train: 0.01091, val: 0.00000
Step:228, train: 0.01107, val: 0.00000
Step:229, train: 0.01104, val: 0.00000
Step:230, train: 0.01067, val: 0.00000
Step:231, train: 0.01059, val: 0.00000
Step:232, train: 0.01074, val: 0.00000
Step:233, train: 0.01054, val: 0.00000
Step:234, train: 0.01054, val: 0.00000
Step:235, train: 0.01093, val: 0.00000
Step:236, train: 0.01074, val: 0.00000
Step:237, train: 0.01059, val: 0.00000
Step:238, train: 0.01048,

Step:424, train: 0.00400, val: 0.00000
Step:425, train: 0.00393, val: 0.00000
Step:426, train: 0.00392, val: 0.00000
Step:427, train: 0.00377, val: 0.00000
Step:428, train: 0.00390, val: 0.00000
Step:429, train: 0.00378, val: 0.00000
Step:430, train: 0.00384, val: 0.00000
Step:431, train: 0.00378, val: 0.00000
Step:432, train: 0.00381, val: 0.00000
Step:433, train: 0.00379, val: 0.00000
Step:434, train: 0.00374, val: 0.00000
Step:435, train: 0.00360, val: 0.00000
Step:436, train: 0.00377, val: 0.00000
Step:437, train: 0.00369, val: 0.00000
Step:438, train: 0.00354, val: 0.00000
Step:439, train: 0.00361, val: 0.00000
Step:440, train: 0.00355, val: 0.00000
Step:441, train: 0.00348, val: 0.00000
Step:442, train: 0.00361, val: 0.00000
Step:443, train: 0.00355, val: 0.00000
Step:444, train: 0.00354, val: 0.00000
Step:445, train: 0.00360, val: 0.00000
Step:446, train: 0.00350, val: 0.00000
Step:447, train: 0.00334, val: 0.00000
Step:448, train: 0.00345, val: 0.00000
Step:449, train: 0.00340,

Step:635, train: 0.00129, val: 0.00000
Step:636, train: 0.00130, val: 0.00000
Step:637, train: 0.00126, val: 0.00000
Step:638, train: 0.00127, val: 0.00000
Step:639, train: 0.00128, val: 0.00000
Step:640, train: 0.00130, val: 0.00000
Step:641, train: 0.00127, val: 0.00000
Step:642, train: 0.00130, val: 0.00000
Step:643, train: 0.00120, val: 0.00000
Step:644, train: 0.00123, val: 0.00000
Step:645, train: 0.00123, val: 0.00000
Step:646, train: 0.00117, val: 0.00000
Step:647, train: 0.00123, val: 0.00000
Step:648, train: 0.00118, val: 0.00000
Step:649, train: 0.00121, val: 0.00000
Step:650, train: 0.00120, val: 0.00000
Step:651, train: 0.00121, val: 0.00000
Step:652, train: 0.00121, val: 0.00000
Step:653, train: 0.00117, val: 0.00000
Step:654, train: 0.00119, val: 0.00000
Step:655, train: 0.00117, val: 0.00000
Step:656, train: 0.00115, val: 0.00000
Step:657, train: 0.00116, val: 0.00000
Step:658, train: 0.00115, val: 0.00000
Step:659, train: 0.00117, val: 0.00000
Step:660, train: 0.00113,

Step:847, train: 0.00047, val: 0.00000
Step:848, train: 0.00048, val: 0.00000
Step:849, train: 0.00047, val: 0.00000
Step:850, train: 0.00046, val: 0.00000
Step:851, train: 0.00046, val: 0.00000
Step:852, train: 0.00046, val: 0.00000
Step:853, train: 0.00045, val: 0.00000
Step:854, train: 0.00047, val: 0.00000
Step:855, train: 0.00046, val: 0.00000
Step:856, train: 0.00045, val: 0.00000
Step:857, train: 0.00045, val: 0.00000
Step:858, train: 0.00046, val: 0.00000
Step:859, train: 0.00045, val: 0.00000
Step:860, train: 0.00045, val: 0.00000
Step:861, train: 0.00045, val: 0.00000
Step:862, train: 0.00046, val: 0.00000
Step:863, train: 0.00045, val: 0.00000
Step:864, train: 0.00044, val: 0.00000
Step:865, train: 0.00044, val: 0.00000
Step:866, train: 0.00043, val: 0.00000
Step:867, train: 0.00042, val: 0.00000
Step:868, train: 0.00042, val: 0.00000
Step:869, train: 0.00043, val: 0.00000
Step:870, train: 0.00042, val: 0.00000
Step:871, train: 0.00043, val: 0.00000
Step:872, train: 0.00043,

In [14]:
channel_fidelity(model_list[0].q_map, kraus_target).numpy()

0.996701631931071

## Retrieve SPAM and Map, with Noise

In [18]:
n = 3
d = 2**n
rank = d
c1 = 0.9
c2 = 0.9

### Retrieve Map without SPAM

In [19]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

model_list = []
for i in range(1):
    
    #make random spam model
    spam_target = SPAM(d=d)

    #add ideal part
    init_target = c1*init_ideal(d) + (1-c1)*spam_target.init
    povm_target = c2*povm_ideal(d) + (1-c2)*spam_target.povm

    spam_target = SPAM(d=d,
                       init = init_target,
                       povm = povm_target)
    
    # random kraus model with unitary part
    kraus_target = KrausMap(
                            U = generate_unitary(d=d), 
                            c = 0.5, 
                            d = d, 
                            rank = rank-1,
                            spam = spam_target
                           )
    
    #inputs_spam, targets_spam = generate_spam_data(spam_target)
    inputs_map, targets_map = generate_map_data(kraus_target, noise = 0.02)
    
    
    kraus_model = KrausMap(
                           d = d, 
                           rank = rank,
                           spam = None,
                           )

    model = ModelQuantumMap(
                            q_map = kraus_model,
                            loss = probs_loss,
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            )

    model.train(inputs = inputs_map,
                targets = targets_map,
                num_iter = 1000,
                N = 1000,
                )
    
    model_list.append(model)

  0%|          | 0/1000 [00:00<?, ?it/s]

Step:0, train: 0.28529, val: 0.00000
Step:1, train: 0.27725, val: 0.00000
Step:2, train: 0.27071, val: 0.00000
Step:3, train: 0.27508, val: 0.00000
Step:4, train: 0.26293, val: 0.00000
Step:5, train: 0.26807, val: 0.00000
Step:6, train: 0.26002, val: 0.00000
Step:7, train: 0.26239, val: 0.00000
Step:8, train: 0.25557, val: 0.00000
Step:9, train: 0.24304, val: 0.00000
Step:10, train: 0.24197, val: 0.00000
Step:11, train: 0.24073, val: 0.00000
Step:12, train: 0.23753, val: 0.00000
Step:13, train: 0.23113, val: 0.00000
Step:14, train: 0.23498, val: 0.00000
Step:15, train: 0.22335, val: 0.00000
Step:16, train: 0.23206, val: 0.00000
Step:17, train: 0.22196, val: 0.00000
Step:18, train: 0.22792, val: 0.00000
Step:19, train: 0.21642, val: 0.00000
Step:20, train: 0.22544, val: 0.00000
Step:21, train: 0.21610, val: 0.00000
Step:22, train: 0.21223, val: 0.00000
Step:23, train: 0.20742, val: 0.00000
Step:24, train: 0.20895, val: 0.00000
Step:25, train: 0.20705, val: 0.00000
Step:26, train: 0.2011

Step:213, train: 0.04334, val: 0.00000
Step:214, train: 0.04350, val: 0.00000
Step:215, train: 0.04445, val: 0.00000
Step:216, train: 0.04405, val: 0.00000
Step:217, train: 0.04440, val: 0.00000
Step:218, train: 0.04330, val: 0.00000
Step:219, train: 0.04366, val: 0.00000
Step:220, train: 0.04289, val: 0.00000
Step:221, train: 0.04303, val: 0.00000
Step:222, train: 0.04271, val: 0.00000
Step:223, train: 0.04389, val: 0.00000
Step:224, train: 0.04363, val: 0.00000
Step:225, train: 0.04274, val: 0.00000
Step:226, train: 0.04239, val: 0.00000
Step:227, train: 0.04237, val: 0.00000
Step:228, train: 0.04227, val: 0.00000
Step:229, train: 0.04364, val: 0.00000
Step:230, train: 0.04356, val: 0.00000
Step:231, train: 0.04244, val: 0.00000
Step:232, train: 0.04217, val: 0.00000
Step:233, train: 0.04256, val: 0.00000
Step:234, train: 0.04251, val: 0.00000
Step:235, train: 0.04215, val: 0.00000
Step:236, train: 0.04193, val: 0.00000
Step:237, train: 0.04290, val: 0.00000
Step:238, train: 0.04288,

Step:425, train: 0.03444, val: 0.00000
Step:426, train: 0.03539, val: 0.00000
Step:427, train: 0.03503, val: 0.00000
Step:428, train: 0.03417, val: 0.00000
Step:429, train: 0.03425, val: 0.00000
Step:430, train: 0.03447, val: 0.00000
Step:431, train: 0.03487, val: 0.00000
Step:432, train: 0.03504, val: 0.00000
Step:433, train: 0.03508, val: 0.00000
Step:434, train: 0.03464, val: 0.00000
Step:435, train: 0.03442, val: 0.00000
Step:436, train: 0.03465, val: 0.00000
Step:437, train: 0.03488, val: 0.00000
Step:438, train: 0.03434, val: 0.00000
Step:439, train: 0.03374, val: 0.00000
Step:440, train: 0.03463, val: 0.00000
Step:441, train: 0.03425, val: 0.00000
Step:442, train: 0.03381, val: 0.00000
Step:443, train: 0.03487, val: 0.00000
Step:444, train: 0.03477, val: 0.00000
Step:445, train: 0.03460, val: 0.00000
Step:446, train: 0.03528, val: 0.00000
Step:447, train: 0.03497, val: 0.00000
Step:448, train: 0.03402, val: 0.00000
Step:449, train: 0.03413, val: 0.00000
Step:450, train: 0.03452,

Step:636, train: 0.03037, val: 0.00000
Step:637, train: 0.03100, val: 0.00000
Step:638, train: 0.03109, val: 0.00000
Step:639, train: 0.03111, val: 0.00000
Step:640, train: 0.03124, val: 0.00000
Step:641, train: 0.03079, val: 0.00000
Step:642, train: 0.03050, val: 0.00000
Step:643, train: 0.03004, val: 0.00000
Step:644, train: 0.02968, val: 0.00000
Step:645, train: 0.03036, val: 0.00000
Step:646, train: 0.03058, val: 0.00000
Step:647, train: 0.03081, val: 0.00000
Step:648, train: 0.03129, val: 0.00000
Step:649, train: 0.03053, val: 0.00000
Step:650, train: 0.03043, val: 0.00000
Step:651, train: 0.03104, val: 0.00000
Step:652, train: 0.03054, val: 0.00000
Step:653, train: 0.03136, val: 0.00000
Step:654, train: 0.03064, val: 0.00000
Step:655, train: 0.03072, val: 0.00000
Step:656, train: 0.03109, val: 0.00000
Step:657, train: 0.03024, val: 0.00000
Step:658, train: 0.03073, val: 0.00000
Step:659, train: 0.03067, val: 0.00000
Step:660, train: 0.03108, val: 0.00000
Step:661, train: 0.03073,

Step:847, train: 0.02841, val: 0.00000
Step:848, train: 0.02804, val: 0.00000
Step:849, train: 0.02897, val: 0.00000
Step:850, train: 0.02807, val: 0.00000
Step:851, train: 0.02827, val: 0.00000
Step:852, train: 0.02894, val: 0.00000
Step:853, train: 0.02801, val: 0.00000
Step:854, train: 0.02808, val: 0.00000
Step:855, train: 0.02812, val: 0.00000
Step:856, train: 0.02824, val: 0.00000
Step:857, train: 0.02854, val: 0.00000
Step:858, train: 0.02850, val: 0.00000
Step:859, train: 0.02765, val: 0.00000
Step:860, train: 0.02800, val: 0.00000
Step:861, train: 0.02857, val: 0.00000
Step:862, train: 0.02823, val: 0.00000
Step:863, train: 0.02922, val: 0.00000
Step:864, train: 0.02846, val: 0.00000
Step:865, train: 0.02876, val: 0.00000
Step:866, train: 0.02810, val: 0.00000
Step:867, train: 0.02888, val: 0.00000
Step:868, train: 0.02814, val: 0.00000
Step:869, train: 0.02820, val: 0.00000
Step:870, train: 0.02814, val: 0.00000
Step:871, train: 0.02830, val: 0.00000
Step:872, train: 0.02841,

In [20]:
channel_fidelity(model_list[0].q_map, kraus_target).numpy()

0.965460694852325

### Retrieve SPAM and Map

In [21]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

model_list = []

for i in range(1):    
    #make random spam model
    spam_target = SPAM(d=d)

    #add ideal part
    init_target = c1*init_ideal(d) + (1-c1)*spam_target.init
    povm_target = c2*povm_ideal(d) + (1-c2)*spam_target.povm

    spam_target = SPAM(d=d,
                       init = init_target,
                       povm = povm_target)
    
    # random kraus model with unitary part
    kraus_target = KrausMap(
                            U = generate_unitary(d=d), 
                            c = 0.5, 
                            d = d, 
                            rank = rank-1,
                            spam = spam_target
                           )
    
    inputs_spam, targets_spam = generate_spam_data(spam_target, noise = 0.02)
    inputs_map, targets_map = generate_map_data(kraus_target, noise = 0.02)

    #train spam model
    spam_model = SPAM(d=d,
    #                  use_corr_mat=True,
                      optimizer = tf.optimizers.Adam(learning_rate=0.01))
    
    spam_model.pretrain(targets = [init_ideal(d), povm_ideal(d)],
                        num_iter = 1000,
                        verbose = False
                        )
    
    spam_model.train(inputs = inputs_spam,
                     targets = targets_spam,
                     num_iter = 1000,
                     verbose = False,
                    )

    #train kraus model
    kraus_model = KrausMap(
                           d = d, 
                           rank = rank,
                           spam = spam_model,
                           )

    model = ModelQuantumMap(
                            q_map = kraus_model,
                            loss = probs_loss,
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            )

    model.train(inputs = inputs_map,
                targets = targets_map,
                num_iter = 1000,
                N = 1000
                )
    
    model_list.append(model)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Step:0, train: 0.24274, val: 0.00000
Step:1, train: 0.24903, val: 0.00000
Step:2, train: 0.24180, val: 0.00000
Step:3, train: 0.23617, val: 0.00000
Step:4, train: 0.23463, val: 0.00000
Step:5, train: 0.23171, val: 0.00000
Step:6, train: 0.23104, val: 0.00000
Step:7, train: 0.22380, val: 0.00000
Step:8, train: 0.23215, val: 0.00000
Step:9, train: 0.22474, val: 0.00000
Step:10, train: 0.22774, val: 0.00000
Step:11, train: 0.22237, val: 0.00000
Step:12, train: 0.21469, val: 0.00000
Step:13, train: 0.21755, val: 0.00000
Step:14, train: 0.20785, val: 0.00000
Step:15, train: 0.20609, val: 0.00000
Step:16, train: 0.20626, val: 0.00000
Step:17, train: 0.21112, val: 0.00000
Step:18, train: 0.20331, val: 0.00000
Step:19, train: 0.19858, val: 0.00000
Step:20, train: 0.19531, val: 0.00000
Step:21, train: 0.19358, val: 0.00000
Step:22, train: 0.19231, val: 0.00000
Step:23, train: 0.18788, val: 0.00000
Step:24, train: 0.19030, val: 0.00000
Step:25, train: 0.18797, val: 0.00000
Step:26, train: 0.1811

Step:215, train: 0.03903, val: 0.00000
Step:216, train: 0.03843, val: 0.00000
Step:217, train: 0.03768, val: 0.00000
Step:218, train: 0.03806, val: 0.00000
Step:219, train: 0.03876, val: 0.00000
Step:220, train: 0.03880, val: 0.00000
Step:221, train: 0.03952, val: 0.00000
Step:222, train: 0.03879, val: 0.00000
Step:223, train: 0.03793, val: 0.00000
Step:224, train: 0.03893, val: 0.00000
Step:225, train: 0.03813, val: 0.00000
Step:226, train: 0.03770, val: 0.00000
Step:227, train: 0.03886, val: 0.00000
Step:228, train: 0.03831, val: 0.00000
Step:229, train: 0.03844, val: 0.00000
Step:230, train: 0.03769, val: 0.00000
Step:231, train: 0.03747, val: 0.00000
Step:232, train: 0.03914, val: 0.00000
Step:233, train: 0.03822, val: 0.00000
Step:234, train: 0.03863, val: 0.00000
Step:235, train: 0.03779, val: 0.00000
Step:236, train: 0.03794, val: 0.00000
Step:237, train: 0.03831, val: 0.00000
Step:238, train: 0.03728, val: 0.00000
Step:239, train: 0.03784, val: 0.00000
Step:240, train: 0.03692,

Step:429, train: 0.03258, val: 0.00000
Step:430, train: 0.03159, val: 0.00000
Step:431, train: 0.03182, val: 0.00000
Step:432, train: 0.03125, val: 0.00000
Step:433, train: 0.03114, val: 0.00000
Step:434, train: 0.03106, val: 0.00000
Step:435, train: 0.03112, val: 0.00000
Step:436, train: 0.03067, val: 0.00000
Step:437, train: 0.03255, val: 0.00000
Step:438, train: 0.03203, val: 0.00000
Step:439, train: 0.03108, val: 0.00000
Step:440, train: 0.03155, val: 0.00000
Step:441, train: 0.03122, val: 0.00000
Step:442, train: 0.03149, val: 0.00000
Step:443, train: 0.03050, val: 0.00000
Step:444, train: 0.03250, val: 0.00000
Step:445, train: 0.03118, val: 0.00000
Step:446, train: 0.03103, val: 0.00000
Step:447, train: 0.03143, val: 0.00000
Step:448, train: 0.03154, val: 0.00000
Step:449, train: 0.03110, val: 0.00000
Step:450, train: 0.03122, val: 0.00000
Step:451, train: 0.03176, val: 0.00000
Step:452, train: 0.03119, val: 0.00000
Step:453, train: 0.03043, val: 0.00000
Step:454, train: 0.03229,

Step:642, train: 0.02863, val: 0.00000
Step:643, train: 0.02919, val: 0.00000
Step:644, train: 0.02864, val: 0.00000
Step:645, train: 0.02956, val: 0.00000
Step:646, train: 0.02922, val: 0.00000
Step:647, train: 0.02783, val: 0.00000
Step:648, train: 0.02855, val: 0.00000
Step:649, train: 0.02934, val: 0.00000
Step:650, train: 0.02969, val: 0.00000
Step:651, train: 0.02866, val: 0.00000
Step:652, train: 0.02873, val: 0.00000
Step:653, train: 0.02897, val: 0.00000
Step:654, train: 0.02878, val: 0.00000
Step:655, train: 0.02895, val: 0.00000
Step:656, train: 0.02977, val: 0.00000
Step:657, train: 0.02969, val: 0.00000
Step:658, train: 0.02893, val: 0.00000
Step:659, train: 0.02826, val: 0.00000
Step:660, train: 0.02945, val: 0.00000
Step:661, train: 0.02957, val: 0.00000
Step:662, train: 0.02919, val: 0.00000
Step:663, train: 0.02910, val: 0.00000
Step:664, train: 0.02878, val: 0.00000
Step:665, train: 0.02913, val: 0.00000
Step:666, train: 0.02837, val: 0.00000
Step:667, train: 0.02903,

Step:855, train: 0.02985, val: 0.00000
Step:856, train: 0.02860, val: 0.00000
Step:857, train: 0.02873, val: 0.00000
Step:858, train: 0.02879, val: 0.00000
Step:859, train: 0.02821, val: 0.00000
Step:860, train: 0.02959, val: 0.00000
Step:861, train: 0.02935, val: 0.00000
Step:862, train: 0.02799, val: 0.00000
Step:863, train: 0.02828, val: 0.00000
Step:864, train: 0.02888, val: 0.00000
Step:865, train: 0.02848, val: 0.00000
Step:866, train: 0.02862, val: 0.00000
Step:867, train: 0.02891, val: 0.00000
Step:868, train: 0.02859, val: 0.00000
Step:869, train: 0.02836, val: 0.00000
Step:870, train: 0.02969, val: 0.00000
Step:871, train: 0.02866, val: 0.00000
Step:872, train: 0.02870, val: 0.00000
Step:873, train: 0.02831, val: 0.00000
Step:874, train: 0.02841, val: 0.00000
Step:875, train: 0.02875, val: 0.00000
Step:876, train: 0.02906, val: 0.00000
Step:877, train: 0.02805, val: 0.00000
Step:878, train: 0.02861, val: 0.00000
Step:879, train: 0.02806, val: 0.00000
Step:880, train: 0.02857,

In [22]:
channel_fidelity(kraus_model, kraus_target).numpy()

0.9571371685833755

### Retrieve SPAM (Corruption Matrix) and Map

In [23]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

model_list = []

for i in range(1):    
    #make random spam model
    spam_target = SPAM(d=d)

    #add ideal part
    init_target = c1*init_ideal(d) + (1-c1)*spam_target.init
    povm_target = c2*povm_ideal(d) + (1-c2)*spam_target.povm

    spam_target = SPAM(d=d,
                       init = init_target,
                       povm = povm_target)
    
    # random kraus model with unitary part
    kraus_target = KrausMap(
                            U = generate_unitary(d=d), 
                            c = 0.5, 
                            d = d, 
                            rank = rank-1,
                            spam = spam_target
                           )
    
    inputs_spam, targets_spam = generate_spam_data(spam_target, noise = 0.02)
    inputs_map, targets_map = generate_map_data(kraus_target, noise = 0.02)

    #train spam model
    spam_model = SPAM(d=d,
                      use_corr_mat=True,
                      optimizer = tf.optimizers.Adam(learning_rate = 0.01))
    
    spam_model.pretrain(targets = [init_ideal(d), povm_ideal(d)],
                        num_iter = 1000,
                        verbose = False
                        )
    
    spam_model.train(inputs = inputs_spam,
                     targets = targets_spam,
                     num_iter = 1000,
                     verbose = False,
                    )

    #train kraus model
    kraus_model = KrausMap(
                           d = d, 
                           rank = rank,
                           spam = spam_model,
                           )

    model = ModelQuantumMap(
                            q_map = kraus_model,
                            loss = probs_loss,
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            )

    model.train(inputs = inputs_map,
                targets = targets_map,
                num_iter = 1000,
                N = 1000
                )
    
    model_list.append(model)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Step:0, train: 0.24209, val: 0.00000
Step:1, train: 0.23631, val: 0.00000
Step:2, train: 0.24181, val: 0.00000
Step:3, train: 0.23482, val: 0.00000
Step:4, train: 0.23440, val: 0.00000
Step:5, train: 0.23333, val: 0.00000
Step:6, train: 0.22988, val: 0.00000
Step:7, train: 0.22791, val: 0.00000
Step:8, train: 0.22247, val: 0.00000
Step:9, train: 0.22814, val: 0.00000
Step:10, train: 0.22261, val: 0.00000
Step:11, train: 0.22099, val: 0.00000
Step:12, train: 0.21514, val: 0.00000
Step:13, train: 0.20936, val: 0.00000
Step:14, train: 0.21587, val: 0.00000
Step:15, train: 0.20819, val: 0.00000
Step:16, train: 0.19993, val: 0.00000
Step:17, train: 0.20147, val: 0.00000
Step:18, train: 0.20364, val: 0.00000
Step:19, train: 0.19996, val: 0.00000
Step:20, train: 0.19410, val: 0.00000
Step:21, train: 0.18794, val: 0.00000
Step:22, train: 0.19177, val: 0.00000
Step:23, train: 0.19292, val: 0.00000
Step:24, train: 0.19239, val: 0.00000
Step:25, train: 0.18751, val: 0.00000
Step:26, train: 0.1882

Step:216, train: 0.03687, val: 0.00000
Step:217, train: 0.03660, val: 0.00000
Step:218, train: 0.03617, val: 0.00000
Step:219, train: 0.03710, val: 0.00000
Step:220, train: 0.03675, val: 0.00000
Step:221, train: 0.03718, val: 0.00000
Step:222, train: 0.03726, val: 0.00000
Step:223, train: 0.03703, val: 0.00000
Step:224, train: 0.03649, val: 0.00000
Step:225, train: 0.03683, val: 0.00000
Step:226, train: 0.03618, val: 0.00000
Step:227, train: 0.03573, val: 0.00000
Step:228, train: 0.03731, val: 0.00000
Step:229, train: 0.03639, val: 0.00000
Step:230, train: 0.03610, val: 0.00000
Step:231, train: 0.03554, val: 0.00000
Step:232, train: 0.03599, val: 0.00000
Step:233, train: 0.03652, val: 0.00000
Step:234, train: 0.03610, val: 0.00000
Step:235, train: 0.03696, val: 0.00000
Step:236, train: 0.03640, val: 0.00000
Step:237, train: 0.03643, val: 0.00000
Step:238, train: 0.03566, val: 0.00000
Step:239, train: 0.03582, val: 0.00000
Step:240, train: 0.03550, val: 0.00000
Step:241, train: 0.03492,

Step:428, train: 0.02936, val: 0.00000
Step:429, train: 0.02887, val: 0.00000
Step:430, train: 0.03018, val: 0.00000
Step:431, train: 0.02912, val: 0.00000
Step:432, train: 0.02953, val: 0.00000
Step:433, train: 0.02921, val: 0.00000
Step:434, train: 0.02941, val: 0.00000
Step:435, train: 0.02881, val: 0.00000
Step:436, train: 0.02901, val: 0.00000
Step:437, train: 0.02857, val: 0.00000
Step:438, train: 0.03001, val: 0.00000
Step:439, train: 0.02932, val: 0.00000
Step:440, train: 0.02876, val: 0.00000
Step:441, train: 0.02871, val: 0.00000
Step:442, train: 0.02881, val: 0.00000
Step:443, train: 0.02933, val: 0.00000
Step:444, train: 0.02816, val: 0.00000
Step:445, train: 0.02946, val: 0.00000
Step:446, train: 0.02853, val: 0.00000
Step:447, train: 0.02869, val: 0.00000
Step:448, train: 0.02874, val: 0.00000
Step:449, train: 0.02877, val: 0.00000
Step:450, train: 0.02875, val: 0.00000
Step:451, train: 0.02877, val: 0.00000
Step:452, train: 0.02911, val: 0.00000
Step:453, train: 0.02926,

Step:640, train: 0.02635, val: 0.00000
Step:641, train: 0.02596, val: 0.00000
Step:642, train: 0.02719, val: 0.00000
Step:643, train: 0.02612, val: 0.00000
Step:644, train: 0.02663, val: 0.00000
Step:645, train: 0.02693, val: 0.00000
Step:646, train: 0.02749, val: 0.00000
Step:647, train: 0.02679, val: 0.00000
Step:648, train: 0.02537, val: 0.00000
Step:649, train: 0.02608, val: 0.00000
Step:650, train: 0.02692, val: 0.00000
Step:651, train: 0.02696, val: 0.00000
Step:652, train: 0.02635, val: 0.00000
Step:653, train: 0.02611, val: 0.00000
Step:654, train: 0.02651, val: 0.00000
Step:655, train: 0.02649, val: 0.00000
Step:656, train: 0.02631, val: 0.00000
Step:657, train: 0.02734, val: 0.00000
Step:658, train: 0.02740, val: 0.00000
Step:659, train: 0.02650, val: 0.00000
Step:660, train: 0.02621, val: 0.00000
Step:661, train: 0.02694, val: 0.00000
Step:662, train: 0.02673, val: 0.00000
Step:663, train: 0.02684, val: 0.00000
Step:664, train: 0.02668, val: 0.00000
Step:665, train: 0.02620,

Step:854, train: 0.02571, val: 0.00000
Step:855, train: 0.02553, val: 0.00000
Step:856, train: 0.02687, val: 0.00000
Step:857, train: 0.02597, val: 0.00000
Step:858, train: 0.02637, val: 0.00000
Step:859, train: 0.02621, val: 0.00000
Step:860, train: 0.02591, val: 0.00000
Step:861, train: 0.02653, val: 0.00000
Step:862, train: 0.02667, val: 0.00000
Step:863, train: 0.02557, val: 0.00000
Step:864, train: 0.02595, val: 0.00000
Step:865, train: 0.02605, val: 0.00000
Step:866, train: 0.02552, val: 0.00000
Step:867, train: 0.02614, val: 0.00000
Step:868, train: 0.02626, val: 0.00000
Step:869, train: 0.02609, val: 0.00000
Step:870, train: 0.02574, val: 0.00000
Step:871, train: 0.02699, val: 0.00000
Step:872, train: 0.02578, val: 0.00000
Step:873, train: 0.02584, val: 0.00000
Step:874, train: 0.02587, val: 0.00000
Step:875, train: 0.02591, val: 0.00000
Step:876, train: 0.02603, val: 0.00000
Step:877, train: 0.02610, val: 0.00000
Step:878, train: 0.02586, val: 0.00000
Step:879, train: 0.02580,

In [24]:
channel_fidelity(kraus_model, kraus_target).numpy()

0.9834880386053395